# Assignment 1

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Reset GPIO to all 0
void reset_gpio(){
    for(int i = 0; i < 8; i++) {
        write_gpio(i,0);
    }
}

In [ ]:
write_gpio(0, 1)
write_gpio(1, 1)
write_gpio(2, 1)
write_gpio(3, 1)

In [ ]:
reset_gpio()

# Assignment 1

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
from pynq.lib import Pmod_IO


In [ ]:
#name pins corresponding to colors
BLUE = Pmod_IO(base.PMODB,1,'out')
GREEN = Pmod_IO(base.PMODB,2,'out')
RED = Pmod_IO(base.PMODB,3,'out')

In [ ]:
def pwm(freq,duty_cycle,pin,t):
    if duty_cycle < 0.01:
        pin.write(0)
        print("Duty cycle <0.01: set to 0%")
        time.sleep(t)
        return True
    if duty_cycle > 0.99:
        pin.write(1)
        print("Duty cycle > 0.99: set to 100%")
        time.sleep(t)
        pin.write(0)
        return True
    period = 1/freq
    t_on = period*duty_cycle
    t_off = period-t_on
    t_end = time.time() + t;
    while time.time() < t_end:
        pin.write(1)
        time.sleep(t_on)
        pin.write(0)
        time.sleep(t_off)
    return True
    

In [ ]:
color = BLUE
pwm(50,0.25,color,3)

In [ ]:
color = BLUE
dc = .01
t = 3
color.write(1)
time.sleep(t)
color.write(0)
pwm(50,dc,color,3)
color.write(1)
time.sleep(t)
color.write(0)

# Part 6: Async

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
from pynq.lib import Pmod_IO

#name pins corresponding to colors
BLUE = Pmod_IO(base.PMODB,1,'out')
GREEN = Pmod_IO(base.PMODB,2,'out')
RED = Pmod_IO(base.PMODB,3,'out')


In [ ]:
def reset_gpio():
    BLUE.write(0)
    GREEN.write(0)
    RED.write(0)

In [ ]:
import asyncio

#power states
ON = 1
OFF = 0

color = RED

cond = True
state = ON

btns = base.btns_gpio

freq = 50 #Hz
duty_cycle = 25 #percent
period = 1/freq
t_on = period*duty_cycle/100
t_off = period - t_on

async def flash_leds():
    global cond, state
    while cond:
        if state == ON:
            for i in range(freq):
                if state == OFF:
                    break
                color.write(1)
                await asyncio.sleep(t_on)
                reset_gpio()
                await asyncio.sleep(t_off)
            await asyncio.sleep(1) #off for 1 sec
        else:
            reset_gpio()
            await asyncio.sleep(0.05)

async def get_btns(_loop):
    global cond, state, color
    while cond:
        await asyncio.sleep(0.01)
        btn = btns.read()
        if btn == 0x01:
            color = RED
            state = ON
            print("red button pressed")
        elif btn == 0x02:
            color = GREEN
            state = ON
            print("green button pressed")
        elif btn == 0x04:
            color = BLUE
            state = ON
            print("blue button pressed")
        elif btn == 0x08:
            reset_gpio()
            state = OFF
            print("off button pressed")
        elif btn > 0x08:
            #press the off button and another at the same time to terminate loop
            reset_gpio()
            state = OFF
            print("Loop Terminated.")
            cond = False
            _loop.stop()

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()        
print("Done.")